In [1]:
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
# Instantiate a spark session to process data
spark = SparkSession.builder.appName("JSON to Data") \
    .config("spark.executor.memory", "6g") \
    .config("spark.driver.memory", "6g") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "4g") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC") \
    .config("spark.driver.extraJavaOptions", "-XX:+UseG1GC") \
    .getOrCreate()

24/09/11 10:17:46 WARN Utils: Your hostname, Randolfs-MacBook-Air.local resolves to a loopback address: 127.0.2.2; using 10.143.60.239 instead (on interface en0)
24/09/11 10:17:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/11 10:17:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.json("../json_data/yelp_academic_dataset_business.json")
df.show()

24/09/11 10:17:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|          city|               hours|is_open|     latitude|     longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|{NULL, NULL, NULL...|Pns2l4eNsfO8kk83d...|Doctors, Traditio...| Santa Barbara|                NULL|      0|   34.4266787|  -119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|87 Grasso Plaza S...|{NULL, NULL, NULL...|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|        Affton|{8:0-18:30, 0:0-0...|      1|   

In [4]:
df.head()

Row(address='1616 Chapala St, Ste 2', attributes=Row(AcceptsInsurance=None, AgesAllowed=None, Alcohol=None, Ambience=None, BYOB=None, BYOBCorkage=None, BestNights=None, BikeParking=None, BusinessAcceptsBitcoin=None, BusinessAcceptsCreditCards=None, BusinessParking=None, ByAppointmentOnly='True', Caters=None, CoatCheck=None, Corkage=None, DietaryRestrictions=None, DogsAllowed=None, DriveThru=None, GoodForDancing=None, GoodForKids=None, GoodForMeal=None, HairSpecializesIn=None, HappyHour=None, HasTV=None, Music=None, NoiseLevel=None, Open24Hours=None, OutdoorSeating=None, RestaurantsAttire=None, RestaurantsCounterService=None, RestaurantsDelivery=None, RestaurantsGoodForGroups=None, RestaurantsPriceRange2=None, RestaurantsReservations=None, RestaurantsTableService=None, RestaurantsTakeOut=None, Smoking=None, WheelchairAccessible=None, WiFi=None), business_id='Pns2l4eNsfO8kk83dixA6A', categories='Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, 

In [5]:
df_ca = df.filter(df['state'] == 'CA')

In [6]:
df.printSchema()

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [7]:
df_ca.createGlobalTempView("CA")

In [8]:
spark.sql("SELECT * FROM global_temp.CA").show()

+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|         city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|{NULL, NULL, NULL...|Pns2l4eNsfO8kk83d...|Doctors, Traditio...|Santa Barbara|                NULL|      0|   34.4266787|   -119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|    827-833 State St|{NULL, NULL, NULL...|noByYNtDLQAra9ccq...|Women's Clothing,...|Santa Barbara|{11:0-18:0, 11:0-...|      1|34.4

In [9]:
spark.sql("SELECT * FROM global_temp.CA WHERE city == 'Santa Barbara' AND is_open == 1").show()
spark.sql("SELECT COUNT(*) FROM global_temp.CA WHERE city = 'Santa Barbara' AND is_open == 1").show()

+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|         city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|    827-833 State St|{NULL, NULL, NULL...|noByYNtDLQAra9ccq...|Women's Clothing,...|Santa Barbara|{11:0-18:0, 11:0-...|      1|34.4202089255|-119.7004597389|                 H&M|      93101|          24|  3.0|   CA|
|131 Anacapa St, S...|{NULL, NULL, u'no...|IDtLPgUrqorrpqSLd...|Food, Restaurants...|Santa Barbara|{8:0-14:0, 0:0-0:...|      1|   3

In [10]:
spark.sql("SELECT * FROM global_temp.CA WHERE city = 'Santa Barbara' AND is_open == 1 AND stars >= 3.9").show()
spark.sql("SELECT COUNT(*) FROM global_temp.CA WHERE city = 'Santa Barbara' AND is_open == 1 AND stars >= 3.9").show()

+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|         city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|131 Anacapa St, S...|{NULL, NULL, u'no...|IDtLPgUrqorrpqSLd...|Food, Restaurants...|Santa Barbara|{8:0-14:0, 0:0-0:...|      1|   34.4144445|   -119.6906718|Helena Avenue Bakery|      93101|         389|  4.0|   CA|
|      825 Cacique St|{NULL, NULL, NULL...|nUqrF-h9S7myCcvND...|Towing, Automotiv...|Santa Barbara|{8:0-17:0, 0:0-0:...|      1|   3

In [11]:
spark.sql("SELECT AVG(stars) FROM global_temp.CA WHERE city = 'Santa Barbara' AND is_open == 1").show()

+-----------------+
|       avg(stars)|
+-----------------+
|4.126490066225165|
+-----------------+



In [12]:
spark.sql("SELECT * FROM global_temp.CA WHERE city = 'Isla Vista' AND is_open == 1 AND stars >= 3.9").show()
spark.sql("SELECT COUNT(*) FROM global_temp.CA WHERE city = 'Isla Vista' AND is_open == 1 AND stars >= 3.9").show()
spark.sql("SELECT AVG(stars) FROM global_temp.CA WHERE city = 'Isla Vista' AND is_open == 1").show()

+--------------------+--------------------+--------------------+--------------------+----------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|      city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+----------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|6530 Pardall Rd, ...|                NULL|2eVDlVkr2uD3wdr6P...|Banks & Credit Un...|Isla Vista|{9:30-16:0, 9:30-...|      1|34.4132270968| -119.854801856|    Wells Fargo Bank|      93117|           9|  4.5|   CA|
|948 Embarcadero D...|{NULL, NULL, NULL...|5RsVAkDnMrcSbErS6...|Food, Internet Ca...|Isla Vista|{7:0-17:30, 7:0-1...|      1|    34.411997|    -

In [13]:
yelp_reviews = spark.read.json("../json_data/yelp_academic_dataset_review.json")

24/09/11 10:17:57 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [14]:
yelp_reviews.createGlobalTempView("reviews")
yelp_reviews.head()

Row(business_id='XQfwVwDr-v0ZS3_CbbE5Xw', cool=0, date='2018-07-07 22:09:11', funny=0, review_id='KU_O5udG6zpxOg-VcAEodg', stars=3.0, text="If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.", useful=0, user_id='mh_-eMZ6K5RLWhZyISBhwA')

In [15]:
spark.sql("SELECT global_temp.CA.business_id, global_temp.CA.city, global_temp.CA.review_count, global_temp.CA.stars, global_temp.CA.name FROM global_temp.CA INNER JOIN global_temp.reviews ON global_temp.reviews.business_id = global_temp.CA.business_id WHERE global_temp.reviews.useful >= 2 AND global_temp.CA.stars >= 3").show()
spark.sql("SELECT COUNT(global_temp.CA.business_id, global_temp.CA.city, global_temp.CA.review_count, global_temp.CA.stars) FROM global_temp.CA INNER JOIN global_temp.reviews ON global_temp.reviews.business_id = global_temp.CA.business_id WHERE global_temp.reviews.useful >= 2 AND global_temp.CA.stars >= 3").show()

+--------------------+-------------+------------+-----+--------------------+
|         business_id|         city|review_count|stars|                name|
+--------------------+-------------+------------+-----+--------------------+
|YbnJYHNp_fHbI-hcF...|Santa Barbara|         195|  5.0|Santa Barbara Adv...|
|18eWJFJbXyR9j_5xf...|  Carpinteria|         460|  4.5|       Siam Elephant|
|pUVRCYL8rT4I5Ry5F...|Santa Barbara|         105|  4.5|Topa Topa Brewing...|
|eL4lyE7LNoXEMvpcJ...|Santa Barbara|         472|  4.0|Deep Sea Tasting ...|
|SZU9c8V2GuREDN5Kg...|Santa Barbara|        2404|  4.0|Santa Barbara She...|
|qdpdUjdkEUpzYDdce...|Santa Barbara|         188|  4.0|          Plaza Deli|
|ujN8QWiLDqNhcu4LQ...|       Goleta|         308|  3.5|Toyota Of Santa B...|
|-Fka99c-tJ-epWYNI...|Santa Barbara|         169|  3.5|         Pizza Mizza|
|NYYPj2WpUuIz2mP6Y...|       Goleta|          98|  4.5|Classic Collision...|
|uI15YInpDsGBUZcNf...|  Carpinteria|         169|  4.0|Borrello's Pizza ...|

+---------------------------------------------+
|count(business_id, city, review_count, stars)|
+---------------------------------------------+
|                                        66092|
+---------------------------------------------+



In [16]:
# Lets explore all restaurants within Santa Barbara
spark.sql("SELECT global_temp.CA.business_id, global_temp.CA.name, global_temp.reviews.text FROM global_temp.reviews INNER JOIN global_temp.CA ON global_temp.CA.business_id = global_temp.CA.business_id WHERE global_temp.CA.city = 'Santa Barbara' AND ',' || categories || ',' LIKE '%,Food,%' OR ',' || categories || ',' LIKE '%,Restaurants,%'").show()
spark.sql("SELECT COUNT(*) FROM global_temp.reviews INNER JOIN global_temp.CA ON global_temp.CA.business_id = global_temp.CA.business_id WHERE global_temp.CA.city = 'Santa Barbara' AND ',' || categories || ',' LIKE '%,Food,%' OR ',' || categories || ',' LIKE '%,Restaurants,%'").show()

+--------------------+--------------------+--------------------+
|         business_id|                name|                text|
+--------------------+--------------------+--------------------+
|IDtLPgUrqorrpqSLd...|Helena Avenue Bakery|If you decide to ...|
|18eWJFJbXyR9j_5xf...|       Siam Elephant|If you decide to ...|
|vLT1KtrA9bWvjFOg-...|   Pieology Pizzeria|If you decide to ...|
|Hqz96v1ymucUKNzIW...|              Subway|If you decide to ...|
|JWFpjvCc_nkNDVtMP...|             Bogo SB|If you decide to ...|
|TZ-bgvsk189WywTLF...|Ascending Health ...|If you decide to ...|
|uFF40n9pOqHK1ciaj...|   Tony's Restaurant|If you decide to ...|
|GgcvRnt5_z3NEC0D6...|   Cafe Buenos Aires|If you decide to ...|
|N-CBZKO37y_WSUN7Z...|Taqueria El Pasto...|If you decide to ...|
|oXP5aohLv5e7R4eah...|B and D's Curbsid...|If you decide to ...|
|bjhCtlYHrkgA5Ku8l...|     Our Daily Bread|If you decide to ...|
|cbkrO0zra1OiNGvtj...|Wok On the Wild Side|If you decide to ...|
|pUVRCYL8rT4I5Ry5F...|Top

+----------+
|  count(1)|
+----------+
|3467178880|
+----------+



In [17]:
santa_barbara_reviews = spark.sql("SELECT global_temp.CA.business_id, global_temp.CA.name, global_temp.reviews.text, global_temp.reviews.stars FROM global_temp.reviews INNER JOIN global_temp.CA ON global_temp.CA.business_id = global_temp.reviews.business_id WHERE global_temp.CA.city = 'Santa Barbara' AND (',' || categories || ',' LIKE '%,Food,%' OR ',' || categories || ',' LIKE '%,Restaurants,%')")
santa_barbara_reviews.show()

+--------------------+--------------------+--------------------+-----+
|         business_id|                name|                text|stars|
+--------------------+--------------------+--------------------+-----+
|IDtLPgUrqorrpqSLd...|Helena Avenue Bakery|What a great addi...|  5.0|
|pUVRCYL8rT4I5Ry5F...|Topa Topa Brewing...|Sweet spot in the...|  5.0|
|bjhCtlYHrkgA5Ku8l...|     Our Daily Bread|Very disappointed...|  1.0|
|IDtLPgUrqorrpqSLd...|Helena Avenue Bakery|Possibly the best...|  5.0|
|k3lTRgvPvYUI-cX7_...|University Club o...|I am so glad we h...|  5.0|
|kZtl_NJjUnlIQgzFr...|             Ichiban|Their sushi is pr...|  3.0|
|IDtLPgUrqorrpqSLd...|Helena Avenue Bakery|Pricey ( a ham an...|  4.0|
|KiE0h68HGOO7ZXAqk...|           Starbucks|I went in when th...|  1.0|
|bjhCtlYHrkgA5Ku8l...|     Our Daily Bread|Great  breads esp...|  5.0|
|XwnkfMUiyH-OsJXbm...|          Sushi Tyme|My go to place fo...|  4.0|
|pUVRCYL8rT4I5Ry5F...|Topa Topa Brewing...|Neat place in the...|  5.0|
|GgcvR

In [18]:
from pyspark.sql.functions import rand
reviews_text = santa_barbara_reviews.orderBy(rand()).limit(35000).select("text", "stars")

In [19]:
reviews_pd = reviews_text.toPandas()

# Cleaning the data
reviews_pd['text'] = reviews_pd['text'].str.lower()
reviews_pd['text'] = reviews_pd['text'].str.replace(r'[^\w\s]', '', regex=True)
reviews_pd = reviews_pd.dropna(subset=['text', 'stars'])

reviews_pd.head()

,text,stars
0,good simple healthy options great big salads,4.0
1,i went to reds last night for a girlfriends bi...,4.0
2,we came here for dinner last night and was abl...,5.0
3,this place has the best ice cream in the world...,5.0
4,place is pricey i tried an espresso donut it w...,3.0


In [20]:
print(reviews_pd['stars'])
print(reviews_pd["text"].value_counts())


0        4.0
1        4.0
2        5.0
3        5.0
4        3.0
        ... 
34995    5.0
34996    5.0
34997    3.0
34998    5.0
34999    4.0
Name: stars, Length: 35000, dtype: float64
text
i love your pizza \n\nis there a chance you guys will ever be available on doordash                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [21]:
from sklearn.utils import resample

df_majority = reviews_pd[reviews_pd['stars'] == 5.0]  # Majority class (or pick the most common class)
df_4 = reviews_pd[reviews_pd['stars'] == 4.0]
df_3 = reviews_pd[reviews_pd['stars'] == 3.0]
df_2 = reviews_pd[reviews_pd['stars'] == 2.0]
df_1 = reviews_pd[reviews_pd['stars'] == 1.0]

# Oversample the minority classes to match the number of samples in the majority class
df_4_oversampled = resample(df_4, replace=True, n_samples=len(df_majority), random_state=42)
df_3_oversampled = resample(df_3, replace=True, n_samples=len(df_majority), random_state=42)
df_2_oversampled = resample(df_2, replace=True, n_samples=len(df_majority), random_state=42)
df_1_oversampled = resample(df_1, replace=True, n_samples=len(df_majority), random_state=42)

# Combine all into a balanced DataFrame
reviews_pd = pd.concat([df_majority, df_4_oversampled, df_3_oversampled, df_2_oversampled, df_1_oversampled])

# Shuffle the dataset to mix up the classes
reviews_pd = reviews_pd.sample(frac=1, random_state=42)

print(reviews_pd['stars'].value_counts())

stars
3.0    18126
1.0    18126
4.0    18126
5.0    18126
2.0    18126
Name: count, dtype: int64


BASIC NAIVE BAYESIAN MODEL FOR SEMANTIC ANALYSIS

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


In [23]:
X = reviews_pd["text"]
y = reviews_pd["stars"].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

print(X_train_vectorized[0])

  (0, 28660)	0.10127126505190344
  (0, 38671)	0.031016308940549114
  (0, 38653)	0.052917178471378994
  (0, 27405)	0.04904461444449384
  (0, 10018)	0.11255030446284403
  (0, 5201)	0.08364145365521758
  (0, 39086)	0.04242408143863115
  (0, 25330)	0.05258729968401357
  (0, 38844)	0.057531079805260905
  (0, 22805)	0.0606113746283814
  (0, 27167)	0.08627587615279732
  (0, 4873)	0.04771840256135236
  (0, 19712)	0.03879423594402194
  (0, 3563)	0.03983015425188597
  (0, 38208)	0.06043994556710097
  (0, 11720)	0.05392432305802477
  (0, 31499)	0.12137452599756592
  (0, 17554)	0.038899916091966635
  (0, 27909)	0.13539839492187575
  (0, 17218)	0.033018316756122344
  (0, 42231)	0.06635285256371438
  (0, 29722)	0.07080334587136011
  (0, 16384)	0.06969787067600994
  (0, 30929)	0.06999621167257987
  (0, 14882)	0.08867111843027113
  :	:
  (0, 28655)	0.06997433484358781
  (0, 41877)	0.16755555154077437
  (0, 42357)	0.04584400486927773
  (0, 39223)	0.08036207770346208
  (0, 22636)	0.1293979527833883
  (0

In [25]:
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

MultinomialNB()

In [26]:
y_pred = model.predict(X_test_vectorized)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.7415866710802163


CREATING LOGISTIC REGRESSION MODEL FOR SEMANTIC ANALYSIS

In [27]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
model.fit(X_train_vectorized, y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [28]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_pred = model.predict(X_test_vectorized)

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2}')

Mean Absolute Error: 0.22536687631027252
Mean Squared Error: 0.3523667659715326
R-squared: 0.8237189765027606


RANDOM FOREST CLASSIFIER

In [29]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'n_estimators': [30, 40, 50],
#     'max_depth': [10, 20],
#     'min_samples_split': [10, 15, 20],
# }

# grid_search = GridSearchCV(RandomForestRegressor(n_jobs=-1, random_state=42), param_grid, cv=5, verbose=2)
# grid_search.fit(X_train_vectorized, y_train)

# print(grid_search.best_params_)

In [30]:
# {'max_depth': 20, 'min_samples_split': 20, 'n_estimators': 50}

# y_pred = grid_search.predict(X_test_vectorized)

# mae = mean_absolute_error(y_test, y_pred)
# print(f'Mean Absolute Error: {mae}')

# mse = mean_squared_error(y_test, y_pred)
# print(f'Mean Squared Error: {mse}')

# r2 = r2_score(y_test, y_pred)
# print(f'R-squared: {r2}')

In [31]:
optimizing_random_forest_model = RandomForestRegressor(max_depth=20, n_estimators=250, min_samples_split=20, verbose=2, n_jobs=-1)
optimizing_random_forest_model.fit(X_train_vectorized, y_train)

building tree 1 of 250
building tree 2 of 250
building tree 3 of 250
building tree 4 of 250
building tree 5 of 250
building tree 6 of 250
building tree 7 of 250
building tree 8 of 250


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 9 of 250
building tree 10 of 250
building tree 11 of 250
building tree 12 of 250
building tree 13 of 250
building tree 14 of 250
building tree 15 of 250
building tree 16 of 250
building tree 17 of 250
building tree 18 of 250
building tree 19 of 250
building tree 20 of 250
building tree 21 of 250
building tree 22 of 250
building tree 23 of 250
building tree 24 of 250
building tree 25 of 250
building tree 26 of 250
building tree 27 of 250building tree 28 of 250

building tree 29 of 250
building tree 30 of 250
building tree 31 of 250
building tree 32 of 250
building tree 33 of 250


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.2min


building tree 34 of 250
building tree 35 of 250
building tree 36 of 250
building tree 37 of 250
building tree 38 of 250
building tree 39 of 250
building tree 40 of 250
building tree 41 of 250
building tree 42 of 250
building tree 43 of 250
building tree 44 of 250
building tree 45 of 250
building tree 46 of 250
building tree 47 of 250
building tree 48 of 250
building tree 49 of 250
building tree 50 of 250
building tree 51 of 250
building tree 52 of 250
building tree 53 of 250
building tree 54 of 250
building tree 55 of 250
building tree 56 of 250
building tree 57 of 250
building tree 58 of 250
building tree 59 of 250
building tree 60 of 250
building tree 61 of 250
building tree 62 of 250
building tree 63 of 250
building tree 64 of 250
building tree 65 of 250
building tree 66 of 250
building tree 67 of 250
building tree 68 of 250
building tree 69 of 250
building tree 70 of 250
building tree 71 of 250
building tree 72 of 250
building tree 73 of 250
building tree 74 of 250
building tree 75

[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  6.2min


building tree 156 of 250
building tree 157 of 250
building tree 158 of 250
building tree 159 of 250
building tree 160 of 250
building tree 161 of 250
building tree 162 of 250
building tree 163 of 250
building tree 164 of 250
building tree 165 of 250
building tree 166 of 250
building tree 167 of 250
building tree 168 of 250
building tree 169 of 250
building tree 170 of 250
building tree 171 of 250
building tree 172 of 250
building tree 173 of 250
building tree 174 of 250
building tree 175 of 250
building tree 176 of 250
building tree 177 of 250
building tree 178 of 250
building tree 179 of 250
building tree 180 of 250
building tree 181 of 250
building tree 182 of 250
building tree 183 of 250
building tree 184 of 250
building tree 185 of 250
building tree 186 of 250
building tree 187 of 250
building tree 188 of 250
building tree 189 of 250
building tree 190 of 250
building tree 191 of 250
building tree 192 of 250
building tree 193 of 250
building tree 194 of 250
building tree 195 of 250


[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 10.2min finished


RandomForestRegressor(max_depth=20, min_samples_split=20, n_estimators=250,
                      n_jobs=-1, verbose=2)

In [32]:
y_pred = optimizing_random_forest_model.predict(X_test_vectorized)

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2}')

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    0.1s


Mean Absolute Error: 0.5435045263899286
Mean Squared Error: 0.5498283619110738
R-squared: 0.7249334621037301


[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.2s finished
